In [ ]:
import osmnx as ox
import pyrosm
import pandas as pd
import geopandas as gpd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from shapely.ops import split
from pyrosm import get_data
from pyrosm.data import sources
from functools import partial
import shapely
from shapely.geometry import Point, LineString, MultiPolygon
from ipyleaflet import Map, GeoJSON, GeoData, basemaps, basemap_to_tiles, Icon, Marker, LayersControl, LayerGroup, DrawControl, FullScreenControl, MeasureControl, ScaleControl, LocalTileLayer

In [ ]:
# fp = get_data("Singapore", update=True)
fp = get_data("London", directory="./data")
print("Data was downloaded to:", fp)

In [ ]:
tanjong_pagar = gpd.read_file('tpzone.geojson')

In [ ]:
tanjong_pagar

## Clean Periphery

In [ ]:
polygon_bbox = tanjong_pagar.geometry.values[0]

In [ ]:
polygon_bbox

In [ ]:
def project_gdf(gdf):
    mean_longitude = gdf["geometry"].representative_point().x.mean()

    # Compute UTM crs
    utm_zone = int(np.floor((mean_longitude + 180) / 6) + 1)
    utm_crs = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

    # project the GeoDataFrame to the UTM CRS
    gdf_proj = gdf.to_crs(utm_crs)
    print(f"Projected to {gdf_proj.crs}")
    
    return gdf_proj

In [ ]:
def buffer_polygon(gdf, bandwidth = 50):
    buffer_zone = project_gdf(gdf).buffer(bandwidth)
    buffered_gdf = buffer_zone.to_crs(4326)
    return buffered_gdf

buffered_tp = buffer_polygon(tanjong_pagar)
buffered_bbox = buffered_tp.geometry.values[0]

In [ ]:
buffered_bbox

In [ ]:
# Initialize the OSM object 
osm = pyrosm.OSM(fp, bounding_box=buffered_bbox)

In [ ]:
nodes, edges = osm.get_network(network_type="driving", nodes=True)

In [ ]:
G_buff = osm.to_graph(nodes, edges, graph_type="networkx", force_bidirectional=True, retain_all=True)

In [ ]:
def great_circle_vec(lat1, lng1, lat2, lng2, earth_radius=6_371_009):
    """
    Calculate great-circle distances between pairs of points.

    Vectorized function to calculate the great-circle distance between two
    points' coordinates or between arrays of points' coordinates using the
    haversine formula. Expects coordinates in decimal degrees.

    Parameters
    ----------
    lat1 : float or numpy.array of float
        first point's latitude coordinate
    lng1 : float or numpy.array of float
        first point's longitude coordinate
    lat2 : float or numpy.array of float
        second point's latitude coordinate
    lng2 : float or numpy.array of float
        second point's longitude coordinate
    earth_radius : float
        earth's radius in units in which distance will be returned (default is
        meters)

    Returns
    -------
    dist : float or numpy.array of float
        distance from each (lat1, lng1) to each (lat2, lng2) in units of
        earth_radius
    """
    y1 = np.deg2rad(lat1)
    y2 = np.deg2rad(lat2)
    dy = y2 - y1

    x1 = np.deg2rad(lng1)
    x2 = np.deg2rad(lng2)
    dx = x2 - x1

    h = np.sin(dy / 2) ** 2 + np.cos(y1) * np.cos(y2) * np.sin(dx / 2) ** 2
    h = np.minimum(1, h)  # protect against floating point errors
    arc = 2 * np.arcsin(np.sqrt(h))

    # return distance in units of earth_radius
    return arc * earth_radius

In [ ]:
# Add edge lengths
def add_edge_lengths(G, precision=3):
    uvk = tuple(G.edges)
    x = G.nodes(data="x")
    y = G.nodes(data="y")
    
    try:
        # two-dimensional array of coordinates: y0, x0, y1, x1
        c = np.array([(y[u], x[u], y[v], x[v]) for u, v, k in uvk])
        
    except KeyError:  # pragma: no cover
        raise KeyError("some edges missing nodes, possibly due to input data clipping issue")
        
    dists = great_circle_vec(c[:, 0], c[:, 1], c[:, 2], c[:, 3]).round(precision)
    dists[np.isnan(dists)] = 0
    nx.set_edge_attributes(G, values=dict(zip(uvk, dists)), name="length")
    
    return G

In [ ]:
G_buff = add_edge_lengths(G_buff)

## Simplify Graph

In [ ]:
def _is_endpoint(G, node, strict=True):
    """
    Is node a true endpoint of an edge.

    Return True if the node is a "real" endpoint of an edge in the network,
    otherwise False. OSM data includes lots of nodes that exist only as points
    to help streets bend around curves. An end point is a node that either:
    1) is its own neighbor, ie, it self-loops.
    2) or, has no incoming edges or no outgoing edges, ie, all its incident
    edges point inward or all its incident edges point outward.
    3) or, it does not have exactly two neighbors and degree of 2 or 4.
    4) or, if strict mode is false, if its edges have different OSM IDs.

    Parameters
    ----------
    G : networkx.MultiDiGraph
        input graph
    node : int
        the node to examine
    strict : bool
        if False, allow nodes to be end points even if they fail all other rules
        but have edges with different OSM IDs

    Returns
    -------
    bool
    """
    neighbors = set(list(G.predecessors(node)) + list(G.successors(node)))
    n = len(neighbors)
    d = G.degree(node)

    # rule 1
    if node in neighbors:
        # if the node appears in its list of neighbors, it self-loops
        # this is always an endpoint.
        return True

    # rule 2
    elif G.out_degree(node) == 0 or G.in_degree(node) == 0:
        # if node has no incoming edges or no outgoing edges, it is an endpoint
        return True

    # rule 3
    elif not (n == 2 and (d == 2 or d == 4)):
        # else, if it does NOT have 2 neighbors AND either 2 or 4 directed
        # edges, it is an endpoint. either it has 1 or 3+ neighbors, in which
        # case it is a dead-end or an intersection of multiple streets or it has
        # 2 neighbors but 3 degree (indicating a change from oneway to twoway)
        # or more than 4 degree (indicating a parallel edge) and thus is an
        # endpoint
        return True

    # rule 4
    elif not strict:
        # non-strict mode: do its incident edges have different OSM IDs?
        osmids = []

        # add all the edge OSM IDs for incoming edges
        for u in G.predecessors(node):
            for key in G[u][node]:
                osmids.append(G.edges[u, node, key]["osmid"])

        # add all the edge OSM IDs for outgoing edges
        for v in G.successors(node):
            for key in G[node][v]:
                osmids.append(G.edges[node, v, key]["osmid"])

        # if there is more than 1 OSM ID in the list of edge OSM IDs then it is
        # an endpoint, if not, it isn't
        return len(set(osmids)) > 1

    # if none of the preceding rules returned true, then it is not an endpoint
    else:
        return False
    
def _build_path(G, endpoint, endpoint_successor, endpoints):
    """
    Build a path of nodes from one endpoint node to next endpoint node.

    Parameters
    ----------
    G : networkx.MultiDiGraph
        input graph
    endpoint : int
        the endpoint node from which to start the path
    endpoint_successor : int
        the successor of endpoint through which the path to the next endpoint
        will be built
    endpoints : set
        the set of all nodes in the graph that are endpoints

    Returns
    -------
    path : list
        the first and last items in the resulting path list are endpoint
        nodes, and all other items are interstitial nodes that can be removed
        subsequently
    """
    # start building path from endpoint node through its successor
    path = [endpoint, endpoint_successor]

    # for each successor of the endpoint's successor
    for successor in G.successors(endpoint_successor):
        if successor not in path:
            # if this successor is already in the path, ignore it, otherwise add
            # it to the path
            path.append(successor)
            while successor not in endpoints:
                # find successors (of current successor) not in path
                successors = [n for n in G.successors(successor) if n not in path]

                # 99%+ of the time there will be only 1 successor: add to path
                if len(successors) == 1:
                    successor = successors[0]
                    path.append(successor)

                # handle relatively rare cases or OSM digitization quirks
                elif len(successors) == 0:
                    if endpoint in G.successors(successor):
                        # we have come to the end of a self-looping edge, so
                        # add first node to end of path to close it and return
                        return path + [endpoint]
                    else:  # pragma: no cover
                        # this can happen due to OSM digitization error where
                        # a one-way street turns into a two-way here, but
                        # duplicate incoming one-way edges are present
                        utils.log(
                            f"Unexpected simplify pattern handled near {successor}", level=lg.WARN
                        )
                        return path
                else:  # pragma: no cover
                    # if successor has >1 successors, then successor must have
                    # been an endpoint because you can go in 2 new directions.
                    # this should never occur in practice
                    raise Exception(f"Unexpected simplify pattern failed near {successor}")

            # if this successor is an endpoint, we've completed the path
            return path

    # if endpoint_successor has no successors not already in the path, return
    # the current path: this is usually due to a digitization quirk on OSM
    return path

def _get_paths_to_simplify(G, strict=True):
    """
    Generate all the paths to be simplified between endpoint nodes.

    The path is ordered from the first endpoint, through the interstitial nodes,
    to the second endpoint.

    Parameters
    ----------
    G : networkx.MultiDiGraph
        input graph
    strict : bool
        if False, allow nodes to be end points even if they fail all other rules
        but have edges with different OSM IDs

    Yields
    ------
    path_to_simplify : list
    """
    # first identify all the nodes that are endpoints
    endpoints = set([n for n in G.nodes if _is_endpoint(G, n, strict=strict)])

    # for each endpoint node, look at each of its successor nodes
    for endpoint in endpoints:
        for successor in G.successors(endpoint):
            if successor not in endpoints:
                # if endpoint node's successor is not an endpoint, build path
                # from the endpoint node, through the successor, and on to the
                # next endpoint node
                yield _build_path(G, endpoint, successor, endpoints)
                
def simplify_graph(G, strict=True, remove_rings=True):
    """
    Simplify a graph's topology by removing interstitial nodes.

    Simplifies graph topology by removing all nodes that are not intersections
    or dead-ends. Create an edge directly between the end points that
    encapsulate them, but retain the geometry of the original edges, saved as
    a new `geometry` attribute on the new edge. Note that only simplified
    edges receive a `geometry` attribute. Some of the resulting consolidated
    edges may comprise multiple OSM ways, and if so, their multiple attribute
    values are stored as a list.

    Parameters
    ----------
    G : networkx.MultiDiGraph
        input graph
    strict : bool
        if False, allow nodes to be end points even if they fail all other
        rules but have incident edges with different OSM IDs. Lets you keep
        nodes at elbow two-way intersections, but sometimes individual blocks
        have multiple OSM IDs within them too.
    remove_rings : bool
        if True, remove isolated self-contained rings that have no endpoints

    Returns
    -------
    G : networkx.MultiDiGraph
        topologically simplified graph, with a new `geometry` attribute on
        each simplified edge
    """
    if "simplified" in G.graph and G.graph["simplified"]:  # pragma: no cover
        raise Exception("This graph has already been simplified, cannot simplify it again.")


    # define edge segment attributes to sum upon edge simplification
    attrs_to_sum = {"length"}

    # make a copy to not mutate original graph object caller passed in
    G = G.copy()
    initial_node_count = len(G)
    initial_edge_count = len(G.edges)
    all_nodes_to_remove = []
    all_edges_to_add = []

    # generate each path that needs to be simplified
    for path in _get_paths_to_simplify(G, strict=strict):

        # add the interstitial edges we're removing to a list so we can retain
        # their spatial geometry
        path_attributes = dict()
        for u, v in zip(path[:-1], path[1:]):

            # there should rarely be multiple edges between interstitial nodes
            # usually happens if OSM has duplicate ways digitized for just one
            # street... we will keep only one of the edges (see below)
            edge_count = G.number_of_edges(u, v)
            if edge_count != 1:
                utils.log(f"Found {edge_count} edges between {u} and {v} when simplifying")

            # get edge between these nodes: if multiple edges exist between
            # them (see above), we retain only one in the simplified graph
            edge_data = G.edges[u, v, 0]
            for attr in edge_data:
                if attr in path_attributes:
                    # if this key already exists in the dict, append it to the
                    # value list
                    path_attributes[attr].append(edge_data[attr])
                else:
                    # if this key doesn't already exist, set the value to a list
                    # containing the one value
                    path_attributes[attr] = [edge_data[attr]]

        # consolidate the path's edge segments' attribute values
        for attr in path_attributes:
            if attr in attrs_to_sum:
                # if this attribute must be summed, sum it now
                path_attributes[attr] = sum(path_attributes[attr])
            elif len(path_attributes[attr]) == 1:
                # if there's only 1 unique value in this attribute list,
                # consolidate it to the single value (the zero-th):
                path_attributes[attr] = path_attributes[attr][0]
            else:
                # otherwise, if there are multiple values, keep one of each
                path_attributes[attr] = tuple(path_attributes[attr])

        # construct the new consolidated edge's geometry for this path
        path_attributes["geometry"] = LineString(
            [Point((G.nodes[node]["x"], G.nodes[node]["y"])) for node in path]
        )

        # add the nodes and edge to their lists for processing at the end
        all_nodes_to_remove.extend(path[1:-1])
        all_edges_to_add.append(
            {"origin": path[0], "destination": path[-1], "attr_dict": path_attributes}
        )

    # for each edge to add in the list we assembled, create a new edge between
    # the origin and destination
    for edge in all_edges_to_add:
        G.add_edge(edge["origin"], edge["destination"], **edge["attr_dict"])

    # finally remove all the interstitial nodes between the new edges
    G.remove_nodes_from(set(all_nodes_to_remove))

    if remove_rings:
        # remove any connected components that form a self-contained ring
        # without any endpoints
        wccs = nx.weakly_connected_components(G)
        nodes_in_rings = set()
        for wcc in wccs:
            if not any(_is_endpoint(G, n) for n in wcc):
                nodes_in_rings.update(wcc)
        G.remove_nodes_from(nodes_in_rings)

    # mark graph as having been simplified
    G.graph["simplified"] = True

    return G

In [ ]:
# Simplify graph, takes in truncated buffer graph, returns the simplified trunc_buffer graph attrs_to_sum = {"length", "travel_time"}

G_buff_simple = simplify_graph(G_buff)

## Truncate polygon

In [ ]:
def graph_to_gdf(G, nodes=False, edges=False, dual=False):

    crs = G.graph['crs']
    if nodes:
        if not G.nodes:  # pragma: no cover
            raise ValueError("graph contains no nodes")
            
        nodes, data = zip(*G.nodes(data=True))
        
        # convert node x/y attributes to Points for geometry column
        geom = (Point(d["x"], d["y"]) for d in data)
        gdf_nodes = gpd.GeoDataFrame(data, index=nodes, crs=crs, geometry=list(geom))
        
        if not dual:
            gdf_nodes.index.rename("osmid", inplace=True)
        
    if edges:
        if not G.edges:  # pragma: no cover
            raise ValueError("graph contains no edges")
        
        if not dual: 
            u, v, k, data = zip(*G.edges(keys=True, data=True))
        else: 
            u, v, data = zip(*G.edges(data=True))
            
        x_lookup = nx.get_node_attributes(G, "x")
        y_lookup = nx.get_node_attributes(G, "y")
        def make_geom(u, v, data, x= x_lookup, y= y_lookup):
            if "geometry" in data:
                return data["geometry"]
            else:
                return LineString((Point((x[u], y[u])), Point((x[v], y[v]))))
            
        geom = map(make_geom, u, v, data)
        gdf_edges = gpd.GeoDataFrame(data, crs=crs, geometry=list(geom))


        # add u, v, key attributes as index
        gdf_edges["u"] = u
        gdf_edges["v"] = v
        if not dual: 
            gdf_edges["key"] = k
            gdf_edges.set_index(["u", "v", "key"], inplace=True)
        else: 
            gdf_edges.set_index(["u", "v"], inplace=True)
    if nodes and edges:
        return gdf_nodes, gdf_edges
    elif nodes:
        return gdf_nodes
    elif edges:
        return gdf_edges

In [ ]:
gs_nodes = graph_to_gdf(G_buff_simple, nodes=True)[["geometry"]]

In [ ]:
# find nodes to keep
to_keep = gs_nodes.within(polygon_bbox)
to_keep = gs_nodes[to_keep]
nodes_outside = gs_nodes[~gs_nodes.index.isin(to_keep.index)]
set_outside = nodes_outside.index
print(f"Nodes in polygon: {len(to_keep)}, nodes outside of polygon: {len(nodes_outside)}.")

In [ ]:
# Truncate by edge
nodes_to_remove = set()
for node in set_outside:
    neighbors = set(G_buff_simple.successors(node)) | set(G_buff_simple.predecessors(node))
    if neighbors.issubset(nodes_outside):
        nodes_to_remove.add(node)

In [ ]:
print(f"Nodes that have no neighbors in polygon: {len(nodes_to_remove)}.")

In [ ]:
G_buff_trunc = G_buff_simple.copy()
initial = G_buff_trunc.number_of_nodes()
G_buff_trunc.remove_nodes_from(nodes_to_remove)
print(f"Initial number of nodes: {initial}, after removal: {G_buff_trunc.number_of_nodes()}")

In [ ]:
max_wcc = max(nx.weakly_connected_components(G_buff_trunc), key=len)
G_buff_trunc = nx.subgraph(G_buff_trunc, max_wcc)
nodes, edges = graph_to_gdf(G_buff_trunc, nodes=True, edges=True)
print(f"Graph has {len(nodes)} nodes and {len(edges)} edges.")

In [ ]:
ax = edges.plot(figsize=(15,15), color="black", alpha = 0.4)
ax = nodes.plot(ax=ax, color="red", markersize=2)

In [ ]:
type(G_buff_trunc)

## Explore Line Graph Option

In [ ]:
osmid_view = nx.get_edge_attributes(G_buff_trunc, "osmid")
osmid_dict = {}
for u,v in set(osmid_view.items()):
    if u not in osmid_dict:
        osmid_dict[(u[:2])] = v
    else: 
        osmid_dict[(u[:2])].append(v)

In [ ]:
length_view = nx.get_edge_attributes(G_buff_trunc, "length")
length_dict = {}
for u,v in set(length_view.items()):
    if u not in length_dict:
        length_dict[(u[:2])] = v
    else: 
        length_dict[(u[:2])].append(v)

In [ ]:
x_dict = nx.get_node_attributes(G_buff_trunc, "x")
y_dict = nx.get_node_attributes(G_buff_trunc, "y")

In [ ]:
L = nx.empty_graph(0)
LG = nx.line_graph(G_buff_trunc)
L.graph['crs'] = 'EPSG:4326'
for node in set(G_buff_trunc.edges()):
    L.add_node(node, length = length_dict[node], osmids = osmid_dict[node], x = (x_dict[node[0]]+x_dict[node[1]])/2, y = (y_dict[node[0]]+y_dict[node[1]])/2, geometry=Point((x_dict[node[0]]+x_dict[node[1]])/2, (y_dict[node[0]]+y_dict[node[1]])/2))
for u,v in set(LG.edges()):
    L.add_edge(u[:2],v[:2])

In [ ]:
L_nodes, L_edges = graph_to_gdf(L, nodes=True, edges=True, dual=True)

In [ ]:
ax = L_edges.plot(figsize=(15,15), color="black", alpha = 0.4)
ax = L_nodes.plot(ax=ax, color="red", markersize=2)

In [ ]:
# Plot edges by some value
# edges['highway'] = pd.Categorical(edges['highway'])
# edges['highway_numeric'] = edges['highway'].cat.codes / 18 * 2
# ax = edges.plot(figsize=(12,12), color="black", alpha = 0.4, linewidth = edges['highway_numeric'])
# ax = nodes.plot(ax=ax, color="red", markersize=2)

## Cluster nodes in large graphs

In [ ]:
G = osm.to_graph(project_gdf(nodes), project_gdf(edges), graph_type="networkx", retain_all=True)

In [ ]:
proj_nodes = project_gdf(nodes)

In [ ]:
def cluster_nodes(proj_gdf, bandwidth = 10):
    cluster = proj_gdf.buffer(bandwidth).unary_union
    cluster_gdf = gpd.GeoDataFrame({'geometry':cluster.geoms}, crs=proj_gdf.crs)
    print(f"Number of nodes before clustering: {len(proj_gdf)}\n Number of nodes after clustering: {len(cluster_gdf)}")
    return cluster_gdf

In [ ]:
cluster_gdf = cluster_nodes(proj_nodes)

In [ ]:
cluster_centroid = cluster_gdf.centroid
cluster_gdf["x"] = cluster_centroid.x
cluster_gdf["y"] = cluster_centroid.y

In [ ]:
merged_gdf = gpd.sjoin(proj_nodes, cluster_gdf, how="left", predicate="within")
merged_gdf = merged_gdf.drop(columns="geometry").rename(columns={"index_right": "cluster"})

In [ ]:
merged_gdf.head()

In [ ]:
clusters = merged_gdf.groupby("cluster")

In [ ]:
for cluster, cluster_nodes in clusters:
    if len(cluster_nodes) > 1:
    # For each cluster and their respective nodes check for sub-clusters
        components = list(nx.weakly_connected_components(G.subgraph(list(cluster_nodes['id']))))
 
    # If there are more than one sub-cluster, merge them into one
    if len(components) > 1:
        sub_label = 0
        for sub_cluster in components:
            sub_cluster_nodes_idx = list(cluster_nodes.index)
            sub_cluster_centroid = proj_nodes.loc[sub_cluster_nodes_idx].unary_union.centroid
            # Assign centroid to each sub-cluster
            merged_gdf.loc[sub_cluster_nodes_idx, "x"] = sub_cluster_centroid.x
            merged_gdf.loc[sub_cluster_nodes_idx, "y"] = sub_cluster_centroid.y
            # Assign new name to each cluster
            merged_gdf.loc[sub_cluster_nodes_idx, "cluster"] = f"{cluster}-{sub_label}"
            sub_label += 1

In [ ]:
merged_gdf["cluster"] = merged_gdf["cluster"].factorize()[0]

In [ ]:
merged_gdf.head()

In [ ]:
N = nx.MultiDiGraph()
N.graph = G.graph

In [ ]:
groups = merged_gdf.groupby("cluster")
for cluster, cluster_nodes in groups:
    node_ids = list(cluster_nodes['id'])
    if len(node_ids) == 1:
        N.add_node(cluster, osmid_original=node_ids[0], x=cluster_nodes["x"].iloc[0], y=cluster_nodes["y"].iloc[0])
    else: 
        N.add_node(
        cluster,
        osmid_original=str(node_ids),
        x=cluster_nodes["x"].iloc[0],
        y=cluster_nodes["y"].iloc[0],
        )


In [ ]:
gdf_edges = graph_to_gdf(G, edges=True)

In [ ]:
merge_index = {}
for k,v in zip(merged_gdf.id, merged_gdf.index):
    merge_index[k] = v

In [ ]:
for u, v, k, data in G.edges(keys=True, data=True):
    u2 = merged_gdf.loc[merge_index[u], "cluster"]
    v2 = merged_gdf.loc[merge_index[v], "cluster"]

    # only create the edge if we're not connecting the cluster
    # to itself, but always add original self-loops
    if (u2 != v2) or (u == v):
        data["u_original"] = u
        data["v_original"] = v
        if "geometry" not in data:
            data["geometry"] = gdf_edges.loc[(u, v, k), "geometry"]
        N.add_edge(u2, v2, **data)

In [ ]:
for cluster, cluster_nodes in groups:

    # but only if there were multiple nodes merged together,
    # otherwise it's the same old edge as in original graph
    if len(cluster_nodes) > 1:

        # get coords of merged nodes point centroid to prepend or
        # append to the old edge geom's coords
        x = N.nodes[cluster]["x"]
        y = N.nodes[cluster]["y"]
        xy = [(x, y)]

        # for each edge incident on this new merged node, update its
        # geometry to extend to/from the new node's point coords
        in_edges = set(N.in_edges(cluster, keys=True))
        out_edges = set(N.out_edges(cluster, keys=True))
        for u, v, k in in_edges | out_edges:
            old_coords = list(N.edges[u, v, k]["geometry"].coords)
            new_coords = xy + old_coords if cluster == u else old_coords + xy
            new_geom = LineString(new_coords)
            N.edges[u, v, k]["geometry"] = new_geom

            # update the edge length attribute, given the new geometry
            N.edges[u, v, k]["length"] = new_geom.length

In [ ]:
new_nodes, new_edges = graph_to_gdf(N, nodes=True, edges=True)

In [ ]:
new_nodes = new_nodes.to_crs(4326)
new_edges = new_edges.to_crs(4326)

In [ ]:
N.remove_nodes_from(list(nx.isolates(N)))

In [ ]:
ax = new_edges.plot(figsize=(12,12), linewidth = 1, color="gray", alpha = 0.5)
ax = new_nodes.plot(ax=ax, color="black", markersize=1)

In [ ]:
buildings = osm.get_buildings()

In [ ]:
proj_buildings = project_gdf(buildings)

In [ ]:
proj_buildings.geometry.plot()

In [ ]:
# Get adjacency based on spatial intersection
building_neighbours = {}
for i,b in zip(proj_buildings.id,proj_buildings.geometry):
    s = proj_buildings.intersects(b.buffer(100))
    building_neighbours[i] = proj_buildings.id[s[s].index].values

In [ ]:
proj_buildings['center'] = proj_buildings.geometry.centroid
proj_buildings = proj_buildings.set_geometry("center")
buildings = proj_buildings.to_crs(4326)
buildings['x'] = buildings.geometry.x
buildings['y'] = buildings.geometry.y

In [ ]:
def great_circle_vec(lat1, lng1, lat2, lng2, earth_radius=6_371_009):
    """
    Calculate great-circle distances between pairs of points.

    Vectorized function to calculate the great-circle distance between two
    points' coordinates or between arrays of points' coordinates using the
    haversine formula. Expects coordinates in decimal degrees.

    Parameters
    ----------
    lat1 : float or numpy.array of float
        first point's latitude coordinate
    lng1 : float or numpy.array of float
        first point's longitude coordinate
    lat2 : float or numpy.array of float
        second point's latitude coordinate
    lng2 : float or numpy.array of float
        second point's longitude coordinate
    earth_radius : float
        earth's radius in units in which distance will be returned (default is
        meters)

    Returns
    -------
    dist : float or numpy.array of float
        distance from each (lat1, lng1) to each (lat2, lng2) in units of
        earth_radius
    """
    y1 = np.deg2rad(lat1)
    y2 = np.deg2rad(lat2)
    dy = y2 - y1

    x1 = np.deg2rad(lng1)
    x2 = np.deg2rad(lng2)
    dx = x2 - x1

    h = np.sin(dy / 2) ** 2 + np.cos(y1) * np.cos(y2) * np.sin(dx / 2) ** 2
    h = np.minimum(1, h)  # protect against floating point errors
    arc = 2 * np.arcsin(np.sqrt(h))

    # return distance in units of earth_radius
    return arc * earth_radius

In [ ]:
buildings = buildings.set_index('id')

In [ ]:
# Get dict of nodes and their attributes
id_to_attributes = {}
for node in set(buildings.index):
    id_to_attributes[node] = buildings.loc[node].to_dict()

In [ ]:
B = nx.empty_graph(0)
B.graph['crs'] = 'EPSG:4326'
for node in set(buildings.index):
    B.add_node(node)

In [ ]:
nx.set_node_attributes(B, id_to_attributes)

In [ ]:
for node, neighbours in building_neighbours.items():
    for neighbour in neighbours:
        B.add_edge(node, neighbour)

In [ ]:
id_to_x = dict(zip(buildings.index, buildings.x))
id_to_y = dict(zip(buildings.index, buildings.y))

In [ ]:
distance_between_buildings = {}
for pair in set(B.edges()):
    distance_between_buildings[pair] = great_circle_vec(id_to_x[pair[0]], id_to_y[pair[0]], id_to_x[pair[1]], id_to_y[pair[1]])

In [ ]:
nx.set_edge_attributes(B, distance_between_buildings, 'length')

In [ ]:
max_wcc = max(nx.connected_components(B), key=len)
B_max = nx.subgraph(B, max_wcc)

In [ ]:
nodes, edges = graph_to_gdf(B_max, nodes=True, edges=True, dual=True)

In [ ]:
ax = edges.plot(figsize=(12,12), linewidth = 0.05, color="black", alpha = 0.8)
# ax = nodes.plot(ax=ax, color="black", markersize=1, alpha=0.3)
# ax = building_poly.plot(ax = ax, color='blue', alpha=1)

In [ ]:
custom_filter = {'amenity': True}
pois = osm.get_pois(custom_filter=custom_filter)

In [ ]:
pois[pois['amenity'] == 'cafe']

In [ ]:
ax = pois.plot(column='amenity', markersize=3, figsize=(12,12), legend=True, legend_kwds=dict(loc='upper left', ncol=5, bbox_to_anchor=(1, 1)))

In [ ]:
landuse = osm.get_landuse()
landuse.plot(column='landuse', legend=True, figsize=(10,6))

In [ ]:
boundaries = osm.get_boundaries(boundary_type="all")

boundaries['geom_type'] = boundaries.geometry.geom_type

# Filter line geometries (something like following should work)
boundaries_lines = boundaries.loc[boundaries["geom_type"].isin(["LineString", "MultiLineString"])]


In [ ]:
tp_zone = boundaries[boundaries['name'] == 'Tanjong Pagar GRC'].to_json()

In [ ]:
with open('tpzone.geojson', 'w') as outfile:
    outfile.write(tp_zone)

In [ ]:
bbox_geom = boundaries[boundaries['name'] == 'Tanjong Pagar GRC']['geometry'].values[0]

In [ ]:
bbox_geom

In [ ]:
type(boundaries[boundaries['name'] == 'Tanjong Pagar GRC']['geometry'])

In [ ]:
osm2 = pyrosm.OSM(fp, bounding_box=bbox_geom)

In [ ]:
tp = osm2.get_buildings()

In [ ]:
tp.building

In [ ]:
ax = tp.plot(column="building", figsize=(12,12), legend=True, legend_kwds=dict(loc='upper left', ncol=3, bbox_to_anchor=(1, 1)))


In [ ]:
walk = osm2.get_network("walking")
walk.plot(color="k", figsize=(12,12), lw=0.7, alpha=0.6)